In [7]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from typing import Optional
from pydantic import BaseModel, Field

In [8]:
base_url = "http://localhost:11434"
model = 'llama3.2:3b'

llm = ChatOllama(base_url=base_url,model=model)

In [2]:
class Joke(BaseModel):
    """Joke to tell user"""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")
    rating: Optional[int] = Field(description="The rating of the joke from 1 to 10")

In [3]:
parser = JsonOutputParser(pydantic_object = Joke)

In [4]:
instruction = parser.get_format_instructions()
print(instruction)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Joke to tell user", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "description": "The rating of the joke from 1 to 10", "title": "Rating"}}, "required": ["setup", "punchline", "rating"]}
```


In [9]:
prompt = PromptTemplate(
    template = ''' 
    Answer the user query with a joke. Here is your formatting instructions.
    {instruction_format}

    Query:{query}
    Answer:''',
    input_variables=['query'],
    partial_variables={'instruction_format': parser.get_format_instructions()}
)

In [10]:
chain = prompt | llm | parser

output = chain.invoke({'query':'Tell me a joke about Marvel Iron Man'})
print(output)

{'setup': 'Tony Stark was feeling stressed at work.', 'punchline': 'So he built an iron stomach.', 'rating': None}
